## Import Libraries 

In [ ]:
import pandas as pd
import numpy as np 

import warnings
warnings.filterwarnings("ignore")


### load data

In [ ]:

train_df = pd.read_csv('/kaggle/input/amazon-reviews/train.csv',header=None, names=['polarity', 'title', 'review'])
train_df.head()

In [ ]:
train_df.sample(5)

### check class imbalance

In [ ]:
train_df.polarity.value_counts()

In [ ]:
# map polarity : positive ->0 | negative -> 1
train_df['label'] = train_df.polarity.map({2:0, 1:1})
train_df.sample(3)

No class imbalance

In [ ]:
# check missing value in review column 
len(train_df[train_df.review.isnull()])

In [ ]:
train_df.review.isnull().any()

In [ ]:
# check missing value in title column 
len(train_df[train_df.title.isnull()])

a 207 NaN value in title, this will create a problem if we planning to combine the title nad review in a single row 

In [ ]:
# combine review title with review body 
# fillna : replaces any NaN values in the title  an empty string 

train_df['full_review'] = train_df.title.fillna('') + " " + train_df.review 

In [ ]:
train_df.head()

In [ ]:
# to check that the new 'full_review' is not affected by the Nan values in title
train_df[train_df.title.isnull()]

In [ ]:
train_df.title[0]

In [ ]:
train_df.review[0]

In [ ]:
train_df.full_review[0]

In [ ]:
# check dublicate 

duplicate_rows = train_df.duplicated() 
duplicate_rows.any() 

No Dublicates in the data 

In [ ]:
len(train_df.full_review[0])

In [ ]:
len(train_df.full_review[100])

In [ ]:
train_df.full_review[100]

In [ ]:
len(train_df.full_review[1])

In [ ]:
# check avergae lenght of a full review 
full_review_avg_length = np.mean([ len(train_df.full_review[0]) for i in range(len(train_df.full_review))])
print(full_review_avg_length) 

In [ ]:
# check for very short reviews 

short_reviews = train_df[train_df['full_review'].apply(lambda x: len(x.split()) < 10)]

short_reviews.label.value_counts()

print(f"Number of short reviews: {len(short_reviews)}")

In [ ]:
short_reviews.head() 

In [ ]:
short_reviews.full_review.iloc[0]

There is a lot of garbage and noise that need our attention 

### a random sample of train_df for ease of experiments

In [ ]:
# a lighter df to experiment with 
df_train_sample  = train_df.sample(100000, random_state=42)

In [ ]:
df_train_sample.label.value_counts() 

In [ ]:
len(df_train_sample)

### Remove URL and HTML tags if any

In [ ]:
from bs4 import BeautifulSoup
has_html_tags = df_train_sample['full_review'].apply(lambda x: bool(BeautifulSoup(x, "html.parser").find(True)))


In [ ]:
df_train_sample[has_html_tags].full_review.iloc[1]

there is indeed html tags that will require our attention 

In [ ]:
# check for urls 
import re
has_url = [bool(re.findall(r'http\S+', df_train_sample.full_review.iloc[i])) for i in range(len(df_train_sample))]
df_train_sample[has_url]

both URL and HTML tags do not affect the feeling of the customer, hence they are just noise in our case.

### Preprocessing Steps:

1. **Remove URLs**: Strips any URLs from the review, as they don't contribute meaningfully to sentiment analysis.

2. **Remove Gibberish & Excessive Repeated Characters**: Identifies and removes sequences with too many repeated characters (e.g., "ooooo" becomes "o") and repeated words (e.g., "great great" becomes "great").

3. **Remove Punctuation, Numbers, and Long Words**: Filters out punctuation, numbers, and excessively long words (over 20 characters).

4. **Lowercase Text**: Converts all text to lowercase.

5. **Remove Stopwords (except "not", "no", and "nor")**: Removes common stopwords except negations to preserve their importance.

6. **Lemmatization**: Converts words to their base form (e.g., "running" to "run").

7. **Word Embeding*


In [ ]:
import re
import spacy
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed



nlp = spacy.load('en_core_web_lg')

# Custom stopwords list to keep "not", "no", "nor"
stopwords_to_keep = {"not", "no", "nor"}
custom_stopwords = nlp.Defaults.stop_words - stopwords_to_keep

# Pre-compile regex patterns for optimization
url_pattern = re.compile(r'http\S+|www\S+|https\S+')
repeated_char_pattern = re.compile(r'(.)\1{3,}')
repeated_word_pattern = re.compile(r'\b(\w+)( \1\b)+')

def remove_gibberish(text):
    # Remove excessive repeated characters 
    text = repeated_char_pattern.sub(r'\1', text)
    # Remove repeated words 
    text = repeated_word_pattern.sub(r'\1', text)
    return text

def preprocess_text(text):
    text = url_pattern.sub('', text)
    text = remove_gibberish(text)
    
    doc = nlp(text)
    
    tokens = [token.lemma_.lower() for token in doc if not (
        token.is_punct or token.is_digit or len(token.text) > 20 or token.text in custom_stopwords)]
    
    clean_text = " ".join(tokens)
    
    # return review vector representation  
    return nlp(clean_text).vector


def combined_preprocessing(text_series):
    # Use parallel processing with joblib to preprocess text in parallel
    results = Parallel(n_jobs=-1, backend="multiprocessing")(
        delayed(preprocess_text)(text) for text in tqdm(text_series, desc="Processing texts")
    )
    return pd.Series(results)


# Function to stack the preprocessed vectors
def stack_preprocessed(preprocessed_series):
    return np.stack(preprocessed_series)


# Custom transformer for stacking
class StackingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return stack_preprocessed(X)


preprocess_pipe = Pipeline([
    ('preprocess', FunctionTransformer(combined_preprocessing, validate=False)),  # Preprocess the text
    ('stacking', StackingTransformer()),
    ('scaling', MinMaxScaler())
])


## Train/Test Split (Using Subset Of Data)

In [ ]:
from sklearn.model_selection import train_test_split

train_df_sample  = train_df.sample(1000, random_state = 42)

X_train,X_test, y_train, y_test = train_test_split(
    train_df_sample.full_review,
    train_df_sample.label,
    random_state=42
)

X_train.shape, X_test.shape


In [ ]:
# test preprocess pipe 
X_train_pre = preprocess_pipe.fit_transform(X_train)
X_test_pre = preprocess_pipe.fit_transform(X_test)

In [ ]:
X_train_pre

## Test that every thing is working

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = make_pipeline(preprocess_pipe, LogisticRegression())

log_reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, log_reg.predict(X_test))) 

## Modeling

- Working with only a random subset of the data.
- The preprocessing step is not in the pipeline to save some time. 

In [ ]:
from sklearn.model_selection import train_test_split

train_df_sample  = train_df.sample(200000, random_state = 42)

X_train,X_test, y_train, y_test = train_test_split(
    train_df_sample.full_review,
    train_df_sample.label,
    test_size=.2,
    random_state=42
)

X_train.shape, X_test.shape

In [ ]:
# To test more models, we will do the preprocessing in a seperate step to save us some time 

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm



X_train_pre = preprocess_pipe.fit_transform(X_train)
X_test_pre = preprocess_pipe.fit_transform(X_test)



In [ ]:
###### Dictionary of models
models = {
    "Logistic Regression": LogisticRegression(solver='lbfgs', max_iter=1000),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "MLP Neural Network": MLPClassifier(max_iter=1000)
}



def evaluate_model(model_name, model, X_train_pre, y_train):
    scores = cross_val_score(model, X_train_pre, y_train, cv=5, scoring='f1_macro', n_jobs=-1) 
    return model_name, scores.mean()

results = []
for model_name, model in tq dm(models.items(), desc="Evaluating models"):
    model_name, score = evaluate_model(model_name, model, X_train_pre, y_train)
    results.append((model_name, score))


In [ ]:
results_df = pd.DataFrame(results, columns=['Model', 'Mean Accuracy'])
results_df


In [ ]:


best_model_name = results_df.loc[results_df['Mean Accuracy'].idxmax(), 'Model']
best_score = results_df['Mean Accuracy'].max()

best_model = models[best_model_name]

print(f"Best model: {best_model_name} with Mean Accuracy: {best_score}")




In [ ]:
best_model.fit(X_train_pre, y_train) 

In [ ]:
# Save the best model using joblib
joblib.dump(best_model, f'best_model_{best_model_name}.pkl')

print(f"The best model {best_model_name} has been saved as 'best_model_{best_model_name}.pkl'.")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib


y_pred = best_model.predict(X_test_pre)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy on test set: {accuracy}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)


## BERT Model Fine-Tuning

In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

### same preprocess as upove but without the word embeding 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import re
import spacy
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed



nlp = spacy.load('en_core_web_lg')

# Custom stopwords list to keep "not", "no", "nor"
stopwords_to_keep = {"not", "no", "nor"}
custom_stopwords = nlp.Defaults.stop_words - stopwords_to_keep

# Pre-compile regex patterns for optimization
url_pattern = re.compile(r'http\S+|www\S+|https\S+')
repeated_char_pattern = re.compile(r'(.)\1{3,}')
repeated_word_pattern = re.compile(r'\b(\w+)( \1\b)+')

def remove_gibberish(text):
    # Remove excessive repeated characters 
    text = repeated_char_pattern.sub(r'\1', text)
    # Remove repeated words 
    text = repeated_word_pattern.sub(r'\1', text)
    return text

def preprocess_text(text):
    text = url_pattern.sub('', text)
    text = remove_gibberish(text)
    
    doc = nlp(text)
    
    tokens = [token.lemma_.lower() for token in doc if not (
        token.is_punct or token.is_digit or len(token.text) > 20 or token.text in custom_stopwords)]
    
    clean_text = " ".join(tokens)
    
    # return review vector representation  
    return clean_text


def combined_preprocessing(text_series):
    # Use parallel processing with joblib to preprocess text in parallel
    results = Parallel(n_jobs=-1, backend="multiprocessing")(
        delayed(preprocess_text)(text) for text in tqdm(text_series, desc="Processing texts")
    )
    return pd.Series(results)


# Function to stack the preprocessed vectors
def stack_preprocessed(preprocessed_series):
    return np.stack(preprocessed_series)


# Custom transformer for stacking
class StackingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return stack_preprocessed(X)


preprocess_pipe = Pipeline([
    ('preprocess', FunctionTransformer(combined_preprocessing, validate=False)),  # Preprocess the text
    #('stacking', StackingTransformer()),
])


In [ ]:
from sklearn.model_selection import train_test_split

train_df_sample  = train_df.sample(150000, random_state = 42)

X_train,X_test, y_train, y_test = train_test_split(
    train_df_sample.full_review,
    train_df_sample.label,
    random_state=42,
    test_size=.15,
)

X_train.shape, X_test.shape

In [ ]:
X_train_pre = preprocess_pipe.fit_transform(X_train)

In [ ]:
X_train_pre = X_train_pre.values
labels = y_train.values 

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
print(f'Original: {X_train_pre[0]}')

print("-"*100)

# The sentence split into tokens
print(f'Tokenized: {tokenizer.tokenize(X_train_pre[0])}')

In [ ]:
# The sentence mapped to token ids 
tokens = tokenizer.tokenize(X_train_pre[0]) # for first instance 

print(f'Token IDs : {tokenizer.convert_tokens_to_ids(tokens)}')

In [ ]:
max_len = 0 

for review in X_train_pre:
    
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(review, add_special_tokens=True)
    
    
    max_len = max(max_len, len(input_ids))

    
print(f'Max Sentence Length {max_len}')

In [ ]:
input_ids = []
attention_masks = [] 
max_len = 200 

# for every review 

for review in X_train_pre:
    
    
    encoded_dict = tokenizer.encode_plus(
        review,                         # review to encode 
        add_special_tokens= True,       # add '[CLS]' and '[SEP]'
        
        max_length= max_len, 
        pad_to_max_length = True,      # pad & Truncate all sentences 
        return_attention_mask= True, 
        
        return_tensors='pt'           # return a pytorch tensor 
    )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    

    
# convert the lists into tensors 

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

labels = torch.tensor(labels) 

In [ ]:
print('Original: ', X_train_pre[20])

print("-"* 100)
print('Token IDs:', input_ids[20])

In [ ]:
dataset = TensorDataset(input_ids, attention_masks, labels)   # every instance is a tuple of 3 


train_size = int(len(dataset) * .8 )
val_size = len(dataset) - train_size 


train_data, val_data = random_split(dataset, [train_size, val_size]) 

print(f'{train_size} training samples') 
print(f'{val_size} validation samples') 


In [ ]:

batch_size = 32 


# train loader 
train_dataloader = DataLoader(
    
    train_data, 
    sampler= RandomSampler(train_data),   # select the batches randomly 
    batch_size = batch_size 

)


# validation loader 
validation_dataloader = DataLoader(
    
    val_data, 
    sampler= RandomSampler(val_data),   # select the batches randomly 
    batch_size = batch_size 

)



In [ ]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 

# with linear classification layer on top 


model = BertForSequenceClassification.from_pretrained(
    
    "bert-base-uncased",
    
    num_labels = 2, 
    
    output_attentions = False, 
    
    output_hidden_states = False,
)



In [ ]:
model = model.to(device)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    

    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)        
        loss = output.loss
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    
    
    
    
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# Loading the best model 

In [ ]:
model = torch.load('bert_model')


In [ ]:
X_test_pre = preprocess_pipe.fit_transform(X_test)
X_test_pre = X_test_pre.values
labels_test = y_test.values 

## Testing

In [ ]:
test_input_ids = []
test_attention_masks = []
for review in X_test_pre:
    encoded_dict = tokenizer.encode_plus(
                        review,                     
                        add_special_tokens = True, 
                        max_length = max_len,           
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])
    
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

In [ ]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            #pred_flat = np.argmax(logits, axis=1).flatten()
            
            predictions.extend(list(logits))

In [ ]:
test_accuracy = flat_accuracy(predictions, labels_test)
print(f'Test accuracy {test_accuracy}') 

In [ ]:
from sklearn.metrics import classification_report 


pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = labels_test.flatten()
print(classification_report(labels_flat, pred_flat))

## Inference 

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

model = torch.load('/kaggle/working/bert_model', map_location=torch.device('cuda'))  
model.eval()  

# Load the same tokenizer used during training.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to preprocess text data (tokenize and create attention masks)
def preprocess_for_bert(text_list, max_len):
    """
    Tokenize and create attention masks for input text.
    Args:
        text_list (list of str): List of input sentences to tokenize.
        max_len (int): Maximum sequence length (should match the max_len used during training).
    Returns:
        input_ids: Tensor of token IDs.
        attention_masks: Tensor of attention masks.
    """
    input_ids = []
    attention_masks = []

    # Tokenize each sentence in the input list
    for text in text_list:
        # Tokenize the input text and add special tokens `[CLS]` and `[SEP]`
        encoded_dict = tokenizer.encode_plus(
            text,                      # Sentence to encode
            add_special_tokens=True,    # Add '[CLS]' and '[SEP]'
            max_length=max_len,         # Pad or truncate the sentence
            pad_to_max_length=True,
            return_attention_mask=True, # Create attention mask
            return_tensors='pt',        # Return PyTorch tensors
        )
        
        # Append the encoded inputs and the attention masks
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert to tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

# Function to perform inference on new text data
def predict(text_list, max_len, device):
    # Step 1: Apply custom preprocessing (tokenization, stopword removal, etc.)
    preprocessed_texts = preprocess_pipe.transform(pd.Series(text_list))
    
    # Step 2: Convert preprocessed text into BERT input format
    input_ids, attention_masks = preprocess_for_bert(preprocessed_texts, max_len)

    # Move inputs to the correct device
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)

    # Perform inference without gradient calculation
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)

    # Extract logits and convert to predicted labels
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).cpu().numpy()

    return predictions




In [ ]:
# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

predictions_mapping = {
    0: "Positive",
    1: "Negative"
}

# Example list of sentences for inference
texts = [
    "This is a Greate Product, i can't wait to try it ",
    "This product is terrible and I am not happy with it."
]

# Define the maximum length for tokenizing (use the same max_len as training)
max_len = 200  

# Get predictions for the input sentences
predictions = predict(texts, max_len, device)

for i, text in enumerate(texts):
    print(f"Text: {text}")
    print(f"Prediction: {predictions_mapping[predictions[i]]}")  # 0 for negative, 1 for positive 
    print("")